In [1]:
import pathlib as pl

import flopy
import matplotlib.pyplot as plt
import xarray as xa
import xugrid as xu

In [2]:
from flopy.mf6.utils import generate_classes
#generate_classes()

# Load the base watershed model

We will use the base watershed model to demonstrate MODFLOW NetCDF input functionality.  

In [3]:
name = "watershed"
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=pl.Path("../../data/watershed/")
)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package npf...
    loading package rch...
    loading package drn...
    loading package drn...
    loading package oc...
  loading solution package model...


# Modify the model to convert ascii input arrays to NetCDF output

We need to set the `start_date_time` variable in the TDIS file and define the `export_netcdf` variable in the groundwater flow model `name_file`. We also need to specifically export array input per package.  In this example we will do this for the DIS package. We will create `structured` NetCDF output first. 

In [4]:
sim.tdis.start_date_time = "1980-01-01"

In [5]:
gwf = sim.get_model()
gwf.export_netcdf = "structured"

In [6]:
gwf.dis.export_array_netcdf = True

# Change the simulation workspace, write the model files, and run the model in validate mode

In [7]:
ws = pl.Path("temp/structured")
sim.set_sim_path(ws)

In [8]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package model...
  writing model model...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package rcha_0...
    writing package river...
    writing package gwd...
    writing package oc...


In [9]:
cargs = ["--mode", "validate"]
sim.run_simulation(cargs=cargs)

FloPy is using the following executable to run the model: ../../../../../../usgs/modflow6/bin/mf6

mf6: MODFLOW 6 simulation mode VALIDATE. Model input will be checked for all
stress periods but the matrix equations will not be assembled or solved.


                               MODFLOW 6 EXTENDED
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                  VERSION 6.6.0.dev0 (preliminary) 05/23/2024
                               ***DEVELOP MODE***

        MODFLOW 6 compiled Aug 27 2024 10:04:53 with GCC version 11.4.0

This software is preliminary or provisional and is subject to 
revision. It is being provided to meet the need for timely best 
science. The software has not received final approval by the U.S. 
Geological Survey (USGS). No warranty, expressed or implied, is made 
by the USGS or the U.S. Government as to the functionality of the 
software and related material nor shall the fact of release 
constitute any such warranty. The software is provided o

(True, [])

# Load the structured NetCDF file with `xarray` and view the contents

In [10]:
nc_ds = xa.open_dataset(ws / "model.nc")

In [11]:
nc_ds

<xarray.Dataset> Size: 267kB
Dimensions:      (z: 5, y: 41, bnd: 2, x: 73)
Coordinates:
  * z            (z) float64 40B 1.0 2.0 3.0 4.0 5.0
  * y            (y) float64 328B 1.012e+05 9.875e+04 ... 3.75e+03 1.25e+03
  * x            (x) float64 584B 1.25e+03 3.75e+03 ... 1.788e+05 1.812e+05
Dimensions without coordinates: bnd
Data variables:
    y_bnds       (y, bnd) float64 656B ...
    x_bnds       (x, bnd) float64 1kB ...
    dis_delr     (x) float64 584B ...
    dis_delc     (y) float64 328B ...
    dis_top      (y, x) float64 24kB ...
    dis_botm     (z, y, x) float64 120kB ...
    dis_idomain  (z, y, x) float64 120kB ...
Attributes:
    title:           MODEL hydraulic head
    source:          MODFLOW 6 6.6.0.dev0 (preliminary) 05/23/2024
    modflow6_grid:   STRUCTURED
    modflow6_model:  MODEL: MODFLOW 6 Groundwater Flow (GWF) model
    history:         first created 2024/8/27 16:28:48.856
    Conventions:     CF-1.11

# Deactivate NetCDF export and activate NetCDF import. Write DIS package that specifies gridded arrays should be read from NetCDF.

In [12]:
# update model for netcdf input
gwf.export_netcdf = None
gwf.nc_filerecord="model.nc"

In [13]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package model...
  writing model model...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package rcha_0...
    writing package river...
    writing package gwd...
    writing package oc...


In [14]:
with open(ws / "model.dis", "w") as f:
    f.write("BEGIN options\n")
    f.write("  XORIGIN       0.00000000\n")
    f.write("  YORIGIN       0.00000000\n")
    f.write("END options\n\n")
    f.write("BEGIN dimensions\n")
    f.write("  NLAY  5\n")
    f.write("  NROW  41\n")
    f.write("  NCOL  73\n")
    f.write("END dimensions\n\n")
    f.write("BEGIN griddata\n")
    f.write("  delr NETCDF\n")
    f.write("  delc NETCDF\n")
    f.write("  top NETCDF\n")
    f.write("  botm NETCDF\n")
    f.write("  idomain NETCDF\n")
    f.write("END griddata\n\n")

In [15]:
sim.run_simulation()

FloPy is using the following executable to run the model: ../../../../../../usgs/modflow6/bin/mf6
                               MODFLOW 6 EXTENDED
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                  VERSION 6.6.0.dev0 (preliminary) 05/23/2024
                               ***DEVELOP MODE***

        MODFLOW 6 compiled Aug 27 2024 10:04:53 with GCC version 11.4.0

This software is preliminary or provisional and is subject to 
revision. It is being provided to meet the need for timely best 
science. The software has not received final approval by the U.S. 
Geological Survey (USGS). No warranty, expressed or implied, is made 
by the USGS or the U.S. Government as to the functionality of the 
software and related material nor shall the fact of release 
constitute any such warranty. The software is provided on the 
condition that neither the USGS nor the U.S. Government shall be held 
liable for any damages resulting from the authorized or unauthorized 
use o

(True, [])

# Modify the model to convert ascii input arrays to NetCDF output

We need modify the `export_netcdf` variable in the groundwater flow model `name_file` to `ugrid` to create mesh NetCDF output. We need to deactivate the NetCDF input specification.

In [16]:
gwf.name_file.export_netcdf = "ugrid"
gwf.nc_filerecord=None

# Change the simulation workspace, write the model files, and run the model in validate mode

In [17]:
ws = pl.Path("temp/mesh")
sim.set_sim_path(ws)

In [18]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package model...
  writing model model...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package rcha_0...
    writing package river...
    writing package gwd...
    writing package oc...


In [19]:
cargs = ["--mode", "validate"]
sim.run_simulation(cargs=cargs)

FloPy is using the following executable to run the model: ../../../../../../usgs/modflow6/bin/mf6

mf6: MODFLOW 6 simulation mode VALIDATE. Model input will be checked for all
stress periods but the matrix equations will not be assembled or solved.


                               MODFLOW 6 EXTENDED
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                  VERSION 6.6.0.dev0 (preliminary) 05/23/2024
                               ***DEVELOP MODE***

        MODFLOW 6 compiled Aug 27 2024 10:04:53 with GCC version 11.4.0

This software is preliminary or provisional and is subject to 
revision. It is being provided to meet the need for timely best 
science. The software has not received final approval by the U.S. 
Geological Survey (USGS). No warranty, expressed or implied, is made 
by the USGS or the U.S. Government as to the functionality of the 
software and related material nor shall the fact of release 
constitute any such warranty. The software is provided o

(True, [])

# Load the mesh NetCDF file with `xugrid` and view the contents

In [20]:
nc_ds = xu.open_dataset(ws / "model.nc")

In [21]:
nc_ds

<xarray.Dataset> Size: 602kB
Dimensions:          (nmesh_node: 3108, nmesh_face: 2993,
                      max_nmesh_face_nodes: 4, x: 73, y: 41)
Coordinates:
  * nmesh_node       (nmesh_node) int64 25kB 0 1 2 3 4 ... 3104 3105 3106 3107
  * nmesh_face       (nmesh_face) int64 24kB 0 1 2 3 4 ... 2989 2990 2991 2992
Dimensions without coordinates: max_nmesh_face_nodes, x, y
Data variables: (12/19)
    mesh_node_x      (nmesh_node) float64 25kB ...
    mesh_node_y      (nmesh_node) float64 25kB ...
    mesh_face_x      (nmesh_face) float64 24kB ...
    mesh_face_xbnds  (nmesh_face, max_nmesh_face_nodes) float64 96kB ...
    mesh_face_y      (nmesh_face) float64 24kB ...
    mesh_face_ybnds  (nmesh_face, max_nmesh_face_nodes) float64 96kB ...
    ...               ...
    dis_botm_l5      (nmesh_face) float64 24kB ...
    dis_idomain_l1   (nmesh_face) float64 24kB ...
    dis_idomain_l2   (nmesh_face) float64 24kB ...
    dis_idomain_l3   (nmesh_face) float64 24kB ...
    dis_idomain_l4   (nmesh_face) float64 24kB ...
    dis_idomain_l5   (nmesh_face) float64 24kB ...
Attributes:
    title:           MODEL hydraulic head
    source:          MODFLOW 6 6.6.0.dev0 (preliminary) 05/23/2024
    modflow6_grid:   LAYERED MESH
    modflow6_model:  MODEL: MODFLOW 6 Groundwater Flow (GWF) model
    history:         first created 2024/8/27 16:28:59.968
    Conventions:     CF-1.11 UGRID-1.0

# Deactivate NetCDF export and activate NetCDF import. Write DIS package that specifies gridded arrays should be read from NetCDF.

In [22]:
# update model for netcdf input
gwf.export_netcdf = None
gwf.nc_filerecord="model.nc"

In [23]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package model...
  writing model model...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package rcha_0...
    writing package river...
    writing package gwd...
    writing package oc...


In [24]:
with open(ws / "model.dis", "w") as f:
    f.write("BEGIN options\n")
    f.write("  XORIGIN       0.00000000\n")
    f.write("  YORIGIN       0.00000000\n")
    f.write("END options\n\n")
    f.write("BEGIN dimensions\n")
    f.write("  NLAY  5\n")
    f.write("  NROW  41\n")
    f.write("  NCOL  73\n")
    f.write("END dimensions\n\n")
    f.write("BEGIN griddata\n")
    f.write("  delr NETCDF\n")
    f.write("  delc NETCDF\n")
    f.write("  top NETCDF\n")
    f.write("  botm NETCDF\n")
    f.write("  idomain NETCDF\n")
    f.write("END griddata\n\n")

In [25]:
sim.run_simulation()

FloPy is using the following executable to run the model: ../../../../../../usgs/modflow6/bin/mf6
                               MODFLOW 6 EXTENDED
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                  VERSION 6.6.0.dev0 (preliminary) 05/23/2024
                               ***DEVELOP MODE***

        MODFLOW 6 compiled Aug 27 2024 10:04:53 with GCC version 11.4.0

This software is preliminary or provisional and is subject to 
revision. It is being provided to meet the need for timely best 
science. The software has not received final approval by the U.S. 
Geological Survey (USGS). No warranty, expressed or implied, is made 
by the USGS or the U.S. Government as to the functionality of the 
software and related material nor shall the fact of release 
constitute any such warranty. The software is provided on the 
condition that neither the USGS nor the U.S. Government shall be held 
liable for any damages resulting from the authorized or unauthorized 
use o

(True, [])